In [ ]:
%pylab inline

In [ ]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.insert(0, '../')
import numpy as np
import cv2
import keras
import pandas as pd
import datetime
from tqdm import tqdm
from glob2 import glob

In [ ]:
from common.utils import *
from common.nn import *
from common.eval import *
import common

In [ ]:
from collections import defaultdict


## Params

In [ ]:
resolutions = [[64*i,192*i] for i in range(1,5)]
boundary_only =  False

In [ ]:
## Data Paths

In [ ]:
DATA_ROOT = '/home/a.gabdullin/datasets/'
TRAIN_DIR = DATA_ROOT+'/SegDataSet_Cityscape_Kitti_Kia_Wilddash/'
im_paths = np.array(glob(TRAIN_DIR + 'resized_image/'  +  '*.*'))
masks_paths = np.array(glob(TRAIN_DIR + 'resized_label/' +  '*.*'))
kia_idx = in_filename(im_paths)
kia_match = np.ones(len(im_paths), dtype=bool)
kia_match[kia_idx] = False
kia_im_paths = im_paths[kia_idx]
kia_mask_paths = masks_paths[kia_idx]
im_paths = im_paths[kia_match]
masks_paths = masks_paths[kia_match]

im_paths_train, im_paths_val, masks_paths_train,masks_paths_val = train_test_split(im_paths, masks_paths,
                                                                                   random_state=42,test_size=0.1)

## Eval Time Perfomance

In [ ]:
total = []

In [ ]:
eval_time = True
if eval_time:
    nets = {}
    total = []
    n_channels=3
    for j in range(len(resolutions)):
        input_shape = resolutions[j]  + [3]
        print(input_shape)
        n_rows,n_cols = resolutions[j]
        im_shape = (n_rows,n_cols)
        X_ = np.array([imread_and_resize(e,im_shape,n_channels == 3) for e in tqdm(im_paths_val,ascii=True)])
        for i in range(3,10):
            nets ={}
            try:
                nets[0] = get_custom_unet(input_shape=input_shape,n_conv=8,bottle_idx=i)
                nets[1] = get_custom_mobunetV1(input_shape=input_shape,n_conv=8,bottle_idx=i)
                nets[2] = get_custom_sepunet(input_shape=input_shape,n_conv=8,bottle_idx=i)
                t0,t1 = inference_time(nets[0],X_),inference_time(nets[1],X_)
                t2 = inference_time(nets[2],X_)
                total.append([input_shape,8,i,t0[1],t1[1],t2[1]])
            except:
                total.append([input_shape,8,i,None,None])

In [ ]:
pd.DataFrame(total,
                 columns=['resolution','n_conv','net-depth','t_unet','t_mobunetV1','t_sepunet'])